## Model Context

A model context supports storage and retrieval of Chat Completion messages.
It is always used together with a model client to generate LLM-based responses.

For example, {py:mod}`~autogen_core.model_context.BufferedChatCompletionContext`
is a most-recent-used (MRU) context that stores the most recent `buffer_size`
number of messages. This is useful to avoid context overflow in many LLMs.

Let's see an example that uses
{py:mod}`~autogen_core.model_context.BufferedChatCompletionContext`.

In [1]:
from jet.logger import logger

Starting cache cleanup thread...
Running cache cleanup...
/Users/jethroestrada/Desktop/External_Projects/Jet_Projects/jet_python_modules/jet/cache/joblib/.cache
Event: pre_start_hook
File: Interactive or unknown context

pre_start_hook triggered at: 2025-04-07|18:03:15


Running cache cleanup...
/Users/jethroestrada/Desktop/External_Projects/Jet_Projects/jet_python_modules/jet/cache/joblib/.cache
Running cache cleanup...
/Users/jethroestrada/Desktop/External_Projects/Jet_Projects/jet_python_modules/jet/cache/joblib/.cache
Running cache cleanup...
/Users/jethroestrada/Desktop/External_Projects/Jet_Projects/jet_python_modules/jet/cache/joblib/.cache
Running cache cleanup...
/Users/jethroestrada/Desktop/External_Projects/Jet_Projects/jet_python_modules/jet/cache/joblib/.cache
Running cache cleanup...
/Users/jethroestrada/Desktop/External_Projects/Jet_Projects/jet_python_modules/jet/cache/joblib/.cache
Running cache cleanup...
/Users/jethroestrada/Desktop/External_Projects/Jet_Projects/jet_python_modules/jet/cache/joblib/.cache
Running cache cleanup...
/Users/jethroestrada/Desktop/External_Projects/Jet_Projects/jet_python_modules/jet/cache/joblib/.cache
Running cache cleanup...
/Users/jethroestrada/Desktop/External_Projects/Jet_Projects/jet_python_modules/

In [2]:
from dataclasses import dataclass

from autogen_core import AgentId, MessageContext, RoutedAgent, SingleThreadedAgentRuntime, message_handler
from autogen_core.model_context import BufferedChatCompletionContext
from autogen_core.models import AssistantMessage, ChatCompletionClient, SystemMessage, UserMessage
from jet.adapters.autogen.ollama_client import OllamaChatCompletionClient

In [3]:
@dataclass
class Message:
    content: str

In [4]:
class SimpleAgentWithContext(RoutedAgent):
    def __init__(self, model_client: ChatCompletionClient) -> None:
        super().__init__("A simple agent")
        self._system_messages = [SystemMessage(content="You are a helpful AI assistant.")]
        self._model_client = model_client
        self._model_context = BufferedChatCompletionContext(buffer_size=5)

    @message_handler
    async def handle_user_message(self, message: Message, ctx: MessageContext) -> Message:
        # Prepare input to the chat completion model.
        user_message = UserMessage(content=message.content, source="user")
        # Add message to model context.
        await self._model_context.add_message(user_message)
        # Generate a response.
        response = await self._model_client.create(
            self._system_messages + (await self._model_context.get_messages()),
            cancellation_token=ctx.cancellation_token,
        )
        # Return with the model's response.
        assert isinstance(response.content, str)
        # Add message to model context.
        await self._model_context.add_message(AssistantMessage(content=response.content, source=self.metadata["type"]))
        return Message(content=response.content)

Now let's try to ask follow up questions after the first one.

In [6]:
model_client = OllamaChatCompletionClient(model="llama3.2")

runtime = SingleThreadedAgentRuntime()
await SimpleAgentWithContext.register(
    runtime,
    "simple_agent_context",
    lambda: SimpleAgentWithContext(model_client=model_client),
)
# Start the runtime processing messages.
runtime.start()
agent_id = AgentId("simple_agent_context", "default")

# First question.
message = Message("Hello, what are some fun things to do in Seattle?")
logger.debug(f"Question: {message.content}")
response = await runtime.send_message(message, agent_id)
logger.gray("-----")

# Second question.
message = Message("What was the first thing you mentioned?")
logger.debug(f"Question: {message.content}")
response = await runtime.send_message(message, agent_id)

# Stop the runtime processing messages.
await runtime.stop()
await model_client.close()

Question: Hello, what are some fun things to do in Seattle?

Calling Ollama chat...
LLM model: llama3.2 (3072) | Tokens: 43


LLM Settings:
temperature: 0.6
num_keep: 0
num_predict: -1
num_ctx: 3072

Stream: True
Model: llama3.2
Prompt Tokens: 43
Max Prompt Tokens: 2816
Remaining Tokens: 2773
num_ctx: 3072
Max Tokens: 3072

Generating response...
Event: call_ollama_chat
File: Interactive or unknown context
Log-Filename: Interactive or unknown context
Seattle is an amazing city with plenty of exciting activities and attractions to explore! Here are some fun things to do in Seattle:

1. **Visit the Space Needle**: Catch panoramic views of the city from the top of the iconic Space Needle.
2. **Explore Pike Place Market**: Walk through the historic market, sample local foods, and watch fishmongers in action.
3. **Walk the Waterfront**: Stroll along the recently revitalized waterfront, enjoy street performers, and take in the stunning views of Puget Sound.
4. **Visit the Chihuly Garden and 

From the second response, you can see the agent now can recall its own previous responses.